In [ ]:
import shutil
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import os
import sys

import plotly.graph_objects as go 
import shutil
import plotly.express as px
import numpy as np 



dir_saveresults='/Users/chiaranotturnogranieri/Downloads/notturno/deseq2/menopause_status/BRCA'
tumor='BRCA'

path_result=os.path.join(dir_saveresults,'result_'+tumor+'.txt')
print(path_result)
df=pd.read_csv(path_result,sep="\t")
display(df)

df['padj']=np.log10(df['padj'])*(-1)
display(df)
#plt
#plotly_volcano(df,dir_saveresults,tumor)


In [ ]:
def plotly_volcano(df):
    fig=go.Figure()
    trace1=go.Scatter(
        x=df['log2FoldChange'],
        y=df['padj'],
        mode='markers',
        hovertext=list(df.index),
       

    )
    fig.add_trace(trace1)
    fig.add_hline(y=-np.log10(0.05),line_dash="dash")
    fig.add_vline(x=1,line_dash="dash")
    fig.add_vline(x=-1,line_dash="dash")
    fig.show()
   # fig.write_html(cartella+'/'+tumor+'.html')



def plotly_volcano(df):
    # Criteri per la colorazione
    #display(df)
    significance_threshold = -np.log10(0.05)
    fold_change_threshold = 1
    
    # Trasformazione del padj con -log10
    

    # Creiamo una colonna 'color' basata sui criteri di significatività
    df['color'] = 'grey'  # default colore

    df.loc[(df['padj'] > significance_threshold), 'color'] = 'blue'  
    df.loc[df['log2FoldChange'] > fold_change_threshold, 'color'] = 'red'  # Up-regolati
    df.loc[df['log2FoldChange'] < -fold_change_threshold, 'color'] = 'red'  # Up-regolati
   
   
    # Creazione del grafico
    fig = go.Figure()

    trace1 = go.Scatter(
        x=df['log2FoldChange'],
        y=df['padj'],
        mode='markers',
        marker=dict(
            color=df['color'],  # Usa la colonna color per colorare i punti
            size=10
        ),
        hovertext=list(df.index),  # Nome dei geni
    )
    
    fig.add_trace(trace1)
    
    # Linee di cutoff per la significatività
    fig.add_hline(y=(significance_threshold), line_dash="dash", line_color="black")  # Padj cutoff
    fig.add_vline(x=fold_change_threshold, line_dash="dash", line_color="black")  # Fold change positivo cutoff
    fig.add_vline(x=-fold_change_threshold, line_dash="dash", line_color="black")  # Fold change negativo cutoff

    # Aggiungere titoli e assi
    fig.update_layout(
        title="Volcano Plot",
        xaxis_title="log2 Fold Change",
        yaxis_title="-log10(padj)"
    )
    
    fig.show()



plotly_volcano(df)

In [1]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
from pathlib import Path
import configparser
from scipy.stats import ranksums
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts
from statsmodels.stats import multitest as multi
import plotly.express as px
import plotly.graph_objects as go 
import shutil

#######################################################################################
#                              Carica il file di configurazione                       #
#######################################################################################

config = configparser.ConfigParser()
#script_dir = Path(__file__).parent

# Costruisci il percorso relativo al file di configurazione
config_file ='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/webserver/conf.ini'

config.read(config_file) 
#directory base
output_data = config['Paths']['output_data']
base_dir = config.get('Paths', 'base_dir', fallback='')

# Costruisci i percorsi completi
def get_full_path(relative_path):
    return os.path.join(base_dir, relative_path)

# Accesso ai valori e costruzione dei percorsi
if 'miRNA' in config:
    miRNA_name = get_full_path(config['miRNA']['name'])
    miRNA_dataframe = get_full_path(config['miRNA']['dataframe'])

if 'gene' in config:
    gene_name_ENSG = get_full_path(config['gene']['name_ENSG'])
    gene_dataframe = get_full_path(config['gene']['dataframe'])
    gene_dataframe_FPKM = get_full_path(config['gene']['dataframe_FPKM'])

if 'protein' in config:
    protein_name = get_full_path(config['protein']['name'])
    protein_dataframe = get_full_path(config['protein']['dataframe'])

if 'clinical' in config:
    clinical_data= get_full_path(config['clinical']['dati_clinici'])
    dati_age= get_full_path(config['clinical']['dati_age'])
    clinical_OS= get_full_path(config['clinical']['clinical_OS'])

if 'os' in config:
    os_pathway=get_full_path(config['os']['os_pathway'])

if 'TCGA' in config:
    TCGA_path=get_full_path(config['tcga']['split_count'])


/Users/chiaranotturnogranieri/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:

def what_is_my_object_gene(gene,control):
    if control=='protein':
        if gene in open(protein_name).read().split("\n"):
            return(gene,'protein','peptide_target',protein_dataframe)
        else:
            return(0) #nome errato
   
    else:
         #implementato per prendere in input anche l'ENSG inserito senza versione.
        df_ensg= pd.read_csv(gene_name_ENSG,sep='\t')
        
        result_index = df_ensg[(df_ensg['gene_id_version'] == gene) | (df_ensg['gene_id'] == gene) | (df_ensg['gene_symbol'] == gene)].index
        if not result_index.empty:
            gene_version=df_ensg.loc[result_index[0],'gene_id_version']

            return (gene_version,'gene','gene_id',int(result_index[0]))
    
        if gene in open(miRNA_name).read().split("\n"):
            return (gene,'miRNA','miRNA_ID',miRNA_dataframe)

        else:
            return(0) #nome errato
    
def open_df_gene(input,tumor,feature,cartella):
    if feature == 'patient_status':
        #print(input)
        posizione="-e "+str(input[3]+2)+"p"
        #creiamo un dataframe piu piccolo dove c'è solo la riga del gene che è stato selezionato
        path=cartella+'/'+str(input[0])+"_df.txt"
        out_file=open(path,"w")
        subprocess.call(["sed","-n", "-e 1p", posizione, gene_dataframe],stdout=out_file)
        return(path)
                
    else:
        path_df="Dataframe_FPKM_"+tumor+".csv"
        path=os.path.join(gene_dataframe_FPKM,path_df)
        return(path)

def open_dataframe(gene,tumor,feature,cartella,control):
    input=what_is_my_object_gene(gene,control)
    #print(input)
    if input!=0:
        if input[1]=='gene':
            #print(feature)
            path=open_df_gene(input,tumor,feature,cartella) #bisogna vedere in base al tipo di feature che viene passata
            df=pd.read_csv(path)
            df=df.set_index(input[2])
            return(df, input[1],input[0])
        else:
            df=df=pd.read_csv(input[3])
            df=df.set_index(input[2])
            return(df,input[1],input[0])
    else: 
        #print("search for the entered name is not available")
        return(0)
    

def plotly_plot(feature,d, gene,cartella,ogg):
        print(list(set(d[feature])))
        fig = px.scatter(x=range(10), y=range(10))
        fig=px.box(d,y=gene,x=feature,color=feature) #points = 'all'
        if ogg[1]== "miRNA":
                fig.update_layout(yaxis_type="log")
        #fig.write_html(cartella+'/'+gene+'_'+feature+'.html')
        fig.show()


def ranksum_test(gene,d,feature,cartella):
    #ranksum test per p-value
    
    p=list(set(d[feature]))
    print(p)


    df1_mask=d[feature]== p[0]
    dp0=d[df1_mask]

    dp0[gene]=dp0[gene].fillna(0)
    display(dp0)
    size_dp0 = len(dp0)

    df1_mask=d[feature]== p[1]
    dp1=d[df1_mask]
    dp1[gene]=dp1[gene].fillna(0)
    size_dp1 = len(dp1)

    w, p_value = ranksums(list(dp0[gene]), list(dp1[gene]))
    print('calcolato',p_value)
    if p_value=='':
        p_value='not significant'
    
    
    # Dimensione dei campioni
    print(len(list(dp0[gene])))
    print(list(dp0[gene]))

    print('#')
    print(len(list(dp1[gene])))
    print(list(dp1[gene]))


    
    
    # Creazione del dizionario dei risultati
    results = {
        "Gene": gene,
        "Feature": feature,
        "Group 1":p[0],
        "Group 2":p[1],
        "Sample Size Group 1": size_dp0,
        "Sample Size Group 2": size_dp1,
        "P-value": p_value
    }
    
    # Creazione di un DataFrame per facilitare l'output
    results_df = pd.DataFrame([results])
    
    # Scrittura su file CSV
    output_file=os.path.join(cartella,'Result.txt')
    display(results_df)
    #results_df.to_csv(output_file, index=False)
    
    return(p_value)





def df_feature(ogg, tumor, feature):
    x=pd.read_csv(clinical_data)
    df1_mask=x['acronym']== tumor
    dfclinical=x[df1_mask]

    if feature=="age_at_initial_pathologic_diagnosis":
       # dfclinical = dfclinical.dropna(subset=['age_at_initial_pathologic_diagnosis'])
        dfclinical[feature] = pd.to_numeric(dfclinical[feature], errors='coerce').astype('Int64')


        median=dfclinical[feature].median()
        
        lista_age=[]
        for ele in dfclinical[feature]:
            
            ele=int(ele)
            if ele<=median:
                lista_age.append("under")
            else:
                lista_age.append("over")
                
        dfclinical=dfclinical.rename(columns={feature:"age"})
        dfclinical[feature]=lista_age

    return (dfclinical)
    

def crealista(dffeat,df,feature):
    #lista di samples di cui abbiamo le features e dati di espressione
    
    #caso specifico se abbiamo feature:patient_status
    if feature=='patient_status':
        lista=list(dffeat['bcr_patient_barcode'])
        listaf01=[]
        listaf=[]
        for ele in df.columns:
            if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
                listaf01.append(ele)

        
        return listaf01

    #altrimenti con tutte le altre feature:
    else:
        lista=list(dffeat['bcr_patient_barcode'])
        
        listaf01=[]
        listaf=[]
        
        for ele in df.columns:
            if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
                if int(ele[-3:-1])<11:
                    if ele[:-4] not in listaf:
                        listaf.append(ele[:-4])
                        listaf01.append(ele)

        if len(listaf)==len(listaf01):
            return listaf, listaf01
        else:
            return 0

import pandas as pd
import os 
import sys
import subprocess


if __name__ == "__main__":
    #parametri in input:
    #gene='ENSG00000242268.2'
    gene='ALPHACATENIN'
    tumor= 'CHOL'
    feature= 'gender'
    # gene='hsa-let-7a-1'
    # tumor= 'ACC'
    # feature= 'gender'
    cartella='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova'
    control='protein'
    ogg=open_dataframe(gene,tumor,feature,cartella,control)
    #print(ogg)
    if ogg!=0:
        gene=ogg[2]
        df=ogg[0]
        dffeat=df_feature(ogg[1], tumor, feature) #df features
        #display(dffeat)
        #remove patient without information abotu feature selected
        dffeat = dffeat.dropna(subset=[feature])
        #display(dffeat)
        if feature== 'patient_status':
            listaf0= crealista(dffeat, df,feature)
            
            lista_exp=[]#lista valori espressione
            lista_feature=[]
            for ele in df[listaf0]:
                lista_exp.append(df.loc[gene,ele])
                if int(ele[-3:-1])<11:
                    lista_feature.append('Tumor')
                else:
                    lista_feature.append('Ctrl')

            #controllo di avere sia dati tumorali che di controllo
            t=0
            c=0
            for ele in lista_feature:
                if ele== 'Tumor':
                    t+=1
                else:
                    c+=1
            if t==0 or c==0:
                print('there is not enough data')               
            
            
            
            d=pd.DataFrame({'nome01':listaf0, gene: lista_exp, feature: lista_feature})
        
        
        else:
            listaf, listaf01= crealista(dffeat, df,feature)
            dffeat=dffeat[['bcr_patient_barcode',feature]]

            #creo un dframe finale
            dframe=pd.DataFrame({'nome01':listaf01, 'bcr_patient_barcode':listaf})
            d= dframe.merge(dffeat)

            lista_exp=[] #lista valori espressione
            for ele in d['nome01']:
                lista_exp.append(df.loc[gene,ele])

            d[gene]=lista_exp

        #plt
        #

        #p-value
        print(ranksum_test(gene,d,feature,cartella))
        #plotly_plot(feature,d, gene,cartella,ogg)
    else:
        print(0)
       
       


['FEMALE', 'MALE']


/var/folders/wp/s7hjh8f11cg7gdw4d_0qdwpc0000gp/T/ipykernel_2267/3747627366.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dp0[gene]=dp0[gene].fillna(0)


,nome01,bcr_patient_barcode,gender,ALPHACATENIN
0,TCGA-ZH-A8Y2-01A,TCGA-ZH-A8Y2,FEMALE,0.0
1,TCGA-W6-AA0S-01A,TCGA-W6-AA0S,FEMALE,0.0
2,TCGA-W5-AA2U-01A,TCGA-W5-AA2U,FEMALE,0.0
3,TCGA-W5-AA2R-01A,TCGA-W5-AA2R,FEMALE,0.0
4,TCGA-ZH-A8Y6-01A,TCGA-ZH-A8Y6,FEMALE,0.0
7,TCGA-W5-AA2Z-01A,TCGA-W5-AA2Z,FEMALE,0.0
8,TCGA-W5-AA36-01A,TCGA-W5-AA36,FEMALE,0.0
9,TCGA-ZD-A8I3-01A,TCGA-ZD-A8I3,FEMALE,0.0
10,TCGA-WD-A7RX-01A,TCGA-WD-A7RX,FEMALE,0.0
11,TCGA-W5-AA2H-01A,TCGA-W5-AA2H,FEMALE,0.0


calcolato 1.0
17
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
#
13
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


/var/folders/wp/s7hjh8f11cg7gdw4d_0qdwpc0000gp/T/ipykernel_2267/3747627366.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dp1[gene]=dp1[gene].fillna(0)


,Gene,Feature,Group 1,Group 2,Sample Size Group 1,Sample Size Group 2,P-value
0,ALPHACATENIN,gender,FEMALE,MALE,17,13,1.0


1.0


In [ ]:
def read_clinical_data():
    x=pd.read_csv(clinical_data)
    x=x.set_index("bcr_patient_barcode")
    return(x)

def what_is_my_object_gene(gene):
    #implementato per prendere in input anche l'ENSG inserito senza versione.
    if 'ENSG' in gene:
        df_ensg= pd.read_csv(gene_name_ENSG,sep='\t')
    
        result_index = df_ensg[(df_ensg['gene_id_version'] == gene) | (df_ensg['gene_id'] == gene)].index
        if not result_index.empty:
            gene_version=df_ensg.loc[result_index[0],'gene_id_version']

            return (gene_version,'gene','gene_id',int(result_index[0]))
        else:
            return(0)

    if gene in open(protein_name).read().split("\n"):
        return(gene,'protein','peptide_target',protein_dataframe)

    if gene in open(miRNA_name).read().split("\n"):
        return (gene,'miRNA','miRNA_ID',miRNA_dataframe)

    else:
        return(0)

def detect_if_gene_mirna_proteina(gene, cartella):
    
    if gene in open(miRNA_name).read().split("\n"):
        os.mkdir(cartella)
        return(['miRNA',"miRNA_ID",miRNA_dataframe])
    
    if gene in open(gene_name_ENSG).read().split("\n"):
        os.mkdir(cartella)
        listageni=open(gene_name_ENSG).read().strip().split("\n")
        posizione="-e "+str(listageni.index(gene)+1)+"p"
        
        #creiamo un dataframe piu piccolo dove c'è solo la riga del gene che è stato selezionato
        path=cartella+'/'+gene+"_df.txt"
        out_file=open(path,"w")
        subprocess.call(["sed","-n", "-e 1p", posizione, gene_dataframe ],stdout=out_file)
        
        return(['gene','gene_id',path])

    if gene in open(protein_name).read().split("\n"):
        os.mkdir(cartella)
        return(['protein', "peptide_target",protein_dataframe])
    
    else:
        return(0) #la ricerca non è disponibile per il nome inserito

def open_dataframe_gene_boxplot_all_tumor(gene,listanomi01, path_dataframe, index):
    if gene == 'miRNA':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index(index)
        return(df)
    if gene == 'gene':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index(index)
        return (df)
    if gene =='protein':
        df=pd.read_csv(path_dataframe,usecols=listanomi01)
        df=df.set_index(index)
        return (df)
    else: 
        print("per il nome inserito non è disponibile la ricerca")
        return 0



def box_plot_all_tumor(df1, cartella, gene, feature):
    
    sns.set_theme(rc={'figure.figsize':(25.7,8.27)})
    sns.set_style("white")

    my_order = df1.groupby(by=["tumor"])[gene].median().iloc[::-1].sort_values().index
    ax=sns.boxplot(x="tumor", y=gene, hue=feature, data=df1, palette="Set2", width=0.7, order=my_order)
    ax.set_yscale("log")
    
    plt.savefig(cartella+'/'+gene+'_'+feature+'.jpg')


def df_feature_age(x,feature):
    if feature=="age_at_initial_pathologic_diagnosis":
        #path_age=os.path.join()
        df=pd.read_csv(dati_age)
        display(df)
        df=df.set_index('acronym')
        
        lista_age=[]
        x1=x
        x1 = x1.drop(x1[x1[feature]== '[Not Available]'].index)

        for tumor,age in zip(x1['acronym'],x1[feature]):
            age=int(age)
            mediana=df.loc[tumor,'median_age']
            
            if age<=mediana:
                lista_age.append("under")
            else:
                lista_age.append("over")

        x1=x1.rename(columns={"age_at_initial_pathologic_diagnosis":"age"})
        x1[feature]=lista_age

        return(x1) 


#per la feature patient status
def crealista_patient_status(dffeat,listamiRNA,ogg):
    lista=list(dffeat.index)
        
    listaf01=[ogg[1]]
    listaf=[]
    for ele in listamiRNA:
        if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
            listaf01.append(ele)
            listaf.append(ele[:-4])
            
    return listaf01,listaf


def pulisci_df(df,feature):
    for tumor in set(list(df.tumor)):
        df1=df
        df1_mask=df['tumor'] == tumor
        df1=df1[df1_mask]

        lista_feature=((list(df1[feature])))
        t=0
        c=0
        for ele in lista_feature:
            if ele== 'Tumor':
                t+=1
            else:
                c+=1       

        #print(tumor, t, c)
        if t==0 or c==0:
            df.drop(df.index[df['tumor'] == tumor], inplace = True)
            
    return df


#ranksum test per p-value
def p_value(df, cartella,feature,gene):
    f=open(cartella+'/result.txt','w')
    f.write('tumor'+'\t'+'p-value'+'\n')
    tumori=set(list(df.tumor))
    for tumor in tumori:
        p=list(set(df[feature]))

        df1=df
        df1_mask=df1['tumor'] == tumor
        df1=df1[df1_mask]


        filtered_df = df1[df1[feature].eq(p[0])]
        list0= list(filtered_df[gene])


        filtered_df = df1[df1[feature].eq(p[1])]
        list1= list(filtered_df[gene])

        w, p = ranksums(list0, list1)
        print(tumor, p)
        f.write(tumor+"\t"+str(p)+"\n")


#################################    
#             MAIN              #
#################################   

if __name__ == "__main__":
    #parametri in input:
    gene= 'ENSG00000242268.2'
    feature= 'age_at_initial_pathologic_diagnosis'
    cartella='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova'


    x=read_clinical_data()
   
    if feature=="age_at_initial_pathologic_diagnosis":
        x1=df_feature_age(x,feature)
    else:
        x1=x


    display(x1)

    # #aprire df
    # ogg_analisi=detect_if_gene_mirna_proteina(gene, cartella) #es. miRNA, miRNA_ID, 'pathdataframe.csv'
    # print(ogg_analisi)
    # if ogg_analisi!=0:
        
    #     #lista samples df expression gene/miRNA/prot
    #     dd=pd.read_csv(ogg_analisi[2],nrows=1)
    #     listamiRNA=list(dd.columns[1:])


    #     #patient status
    #     if feature== 'patient_status':    
    #         listanomi01, listanomi= crealista_patient_status(x, listamiRNA,ogg_analisi)
            
    #         #aprire df espressione
    #         d=open_dataframe_gene_boxplot_all_tumor(ogg_analisi[0],listanomi01, ogg_analisi[2],ogg_analisi[1])
            

    #         lista_exp=[]#lista valori espressione
    #         lista_feature=[]#lista valori feature
    #         for ele in d.columns:
    #             lista_exp.append(d.loc[gene,ele])
    #             if int(ele[-3:-1])<11:
    #                 lista_feature.append('Tumor')
    #             else:
    #                 lista_feature.append('Ctrl')
                    

    #     else:
    #         #list samples with clinical data
    #         listap=list(x1.index) 
            
    #         listanomi=[] #nomi con cui ricavare i dati clinici
    #         listanomi01=[ogg_analisi[1]] #nomi con cui ricavare dati di espressione
    #         for el in listamiRNA:
    #             if el[:-4] in listap and el[:-4] not in listanomi:
    #                 if int(el[-3:-1])<11:
    #                     listanomi.append(el[:-4])
    #                     listanomi01.append(el)


    #         #df espressione solo dei campioni di cui abbiamo i dati clinici
    #         print(ogg_analisi[2])
    #         d=open_dataframe_gene_boxplot_all_tumor(ogg_analisi[0],listanomi01, ogg_analisi[2])
    #         print('here')


    #     colonna1=list(d.loc[gene,])
    #     colonna2=list(x1.loc[listanomi, "acronym"])
    #     if feature== 'patient_status':
    #         colonna3=lista_feature
    #     else:
    #         colonna3=list(x1.loc[listanomi, feature])


    #     df=pd.DataFrame({gene:colonna1, 'tumor':colonna2, feature:colonna3})
    #     df=df.dropna()


    #     #rimuovere i tumori che non hanno il ctrl 
    #     if feature=='patient_status':
    #         df=pulisci_df(df,feature)  

    #     box_plot_all_tumor(df, cartella, gene, feature)
    #     p_value(df, cartella,feature,gene)

    # else:
    #     print('Ricerca non disponibile per il nome inserito')

In [2]:
#prova demut volcano plot
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from decimal import Decimal
import sys
import os
import numpy as np
from function_new_prova import mapping_ensg
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
from pathlib import Path
import configparser
from scipy.stats import ranksums
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts
from statsmodels.stats import multitest as multi
import plotly.express as px
import plotly.graph_objects as go 
import shutil
#from statannotations.Annotator import Annotator
from decimal import Decimal

def plotly_volcano(df,cartella,tumor):

    significance_threshold = -np.log10(0.05)
    fold_change_threshold = 1
    df['color'] = 'grey'  # default colore
    df.loc[(df['padj'] > significance_threshold), 'color'] = 'grey'  
    df.loc[(df['padj'] < significance_threshold), 'color'] = 'grey'  
    df.loc[(df['log2FoldChange'] >= fold_change_threshold) & (df['padj'] > significance_threshold), 'color'] = 'red' # Up-regolati
    df.loc[(df['log2FoldChange'] <= -fold_change_threshold) & (df['padj'] > significance_threshold), 'color'] = 'blue'  # Down-regolati
   
    fig=go.Figure()
    trace1=go.Scatter(
        x=df['log2FoldChange'],
        y=df['padj'],
        mode='markers',
        hovertext=list(df['GeneSymbol']),
        marker=dict(
            color=df['color'],  # Usa la colonna color per colorare i punti
            size=10
        ),
    )
    fig.add_trace(trace1)
    fig.add_hline(y=(significance_threshold),line_dash="dash")# Padj cutoff
    fig.add_vline(x=fold_change_threshold, line_dash="dash")# line_color="black")  # Fold change positivo cutoff
    fig.add_vline(x=-fold_change_threshold, line_dash="dash")#, line_color="black")  # Fold change negativo cutoff

    fig.update_layout(
        title="Volcano Plot",
        xaxis_title="log2 Fold Change",
        yaxis_title="-log10(padj)"
    )
    #fig.write_html(cartella+'/'+tumor+'.html')
    fig.show()

if __name__ == "__main__":
    # tumor= sys.argv[1]
    # file_result= sys.argv[2] #"/mnt/data/notturno/deseq2/tobacco_smoking_history/BLCA/result_BLCA.txt"
    # dir_saveresults=sys.argv[3]
    tumor='CHOL'
    file_result= "res_CHOL_KRAS.txt"
    dir_saveresults="/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova/"
    #generare file html plotly per volcano plot nella cartella risultati
    path_result=os.path.join(dir_saveresults,file_result)
    print(path_result)
    df=pd.read_csv(path_result,sep="\t",header=None,skiprows=1)

    df.columns=["ENSG","baseMean","log2FoldChange","lfcSE","stat","pvalue","padj"]

    file_mapped=mapping_ensg()
    ens2sym=pd.read_csv(file_mapped,sep="\t",index_col=0)
    

    GS=[]
    for i in df.ENSG:
        if i in ens2sym.index:
            GS.append(ens2sym.loc[i,"gene_symbol"])
        else:
            GS.append(i)


    df["GeneSymbol"]=GS
    col=["GeneSymbol","ENSG","log2FoldChange","padj"]

    df=df.loc[:,col].set_index("ENSG")
  
    df.log2FoldChange=[round(x, 3) for x in df.log2FoldChange]

    #df.to_csv(os.path.join(dir_saveresults,'result.txt'), sep="\t")

    display(df.head())
    print('################')
    #df_res=pd.read_csv(os.path.join(dir_saveresults,'result.txt'),sep="\t")
   # display(df_res.head())
    
    df_plot=df.copy()
    df_plot['padj']=np.log10(df_plot['padj'])*(-1)
    #df_plot.to_csv(os.path.join(dir_saveresults,'result_-log.txt'), sep="\t")
    display(df_plot.head())
    plotly_volcano(df_plot,dir_saveresults,tumor)
    

    # df_res.padj=['%.2E' % Decimal(x) for x in df_res.padj]
    # display(df_res)

    #df.padj=['%.2E' % Decimal(x) for x in df.padj]
    display(df)


    display(df.sort_values(by='padj', ascending=True).head(10))
    # #plt
    # df_plot=df
    # #df_plot['padj']=np.log10(df_plot['padj'])*(-1)
    # df['padj']=np.log10(df['padj'])*(-1)
    # plotly_volcano(df,dir_saveresults,tumor)

    # #sovrascrive il file di risultati
    # df.padj=['%.2E' % Decimal(x) for x in df.padj]
     
    
    # #df.to_csv(path_result,sep="\t")
    # #df.to_csv(os.path.join(dir_saveresults,'result.txt'), sep="\t")






/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova/res_CHOL_KRAS.txt


,GeneSymbol,log2FoldChange,padj
ENSG,,,
ENSG00000000003.13,TSPAN6,-0.371,0.946886
ENSG00000000419.11,DPM1,0.055,0.984767
ENSG00000000457.12,SCYL3,-0.453,0.885022
ENSG00000000460.15,C1orf112,0.593,0.754551
ENSG00000000938.11,FGR,-1.369,0.550424


################


,GeneSymbol,log2FoldChange,padj
ENSG,,,
ENSG00000000003.13,TSPAN6,-0.371,0.023702
ENSG00000000419.11,DPM1,0.055,0.006666
ENSG00000000457.12,SCYL3,-0.453,0.053046
ENSG00000000460.15,C1orf112,0.593,0.122311
ENSG00000000938.11,FGR,-1.369,0.259303


,GeneSymbol,log2FoldChange,padj
ENSG,,,
ENSG00000000003.13,TSPAN6,-0.371,0.946886
ENSG00000000419.11,DPM1,0.055,0.984767
ENSG00000000457.12,SCYL3,-0.453,0.885022
ENSG00000000460.15,C1orf112,0.593,0.754551
ENSG00000000938.11,FGR,-1.369,0.550424
...,...,...,...
ENSG00000281599.1,AC023283.1,-0.319,NaN
ENSG00000281649.1,EBLN3,0.063,0.991023
ENSG00000281706.1,LINC01012,1.318,0.711830


,GeneSymbol,log2FoldChange,padj
ENSG,,,
ENSG00000189120.4,SP6,6.744,1.071399e-14
ENSG00000111581.8,NUP107,3.119,1.272717e-10
ENSG00000123427.14,METTL21B,3.417,1.290778e-10
ENSG00000115828.14,QPCT,5.679,4.241844e-10
ENSG00000111432.4,FZD10,6.134,2.016488e-09
ENSG00000111605.15,CPSF6,2.146,2.380927e-09
ENSG00000185269.10,NOTUM,10.094,1.210949e-08
ENSG00000037897.15,METTL1,3.234,1.754494e-08
ENSG00000159784.16,FAM131B,4.931,5.915172e-08


In [ ]:
#####
#dash


In [ ]:
#prova demut volcano plot
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from decimal import Decimal
import sys
import os
import numpy as np
from function_new_prova import mapping_ensg
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
from pathlib import Path
import configparser
from scipy.stats import ranksums
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts
from statsmodels.stats import multitest as multi
import plotly.express as px
import plotly.graph_objects as go 
import shutil
#from statannotations.Annotator import Annotator
from decimal import Decimal

def plotly_volcano(df,cartella,tumor):

    significance_threshold = -np.log10(0.05)
    fold_change_threshold = 1
    df['color'] = 'grey'  # default colore
    df.loc[(df['padj'] > significance_threshold), 'color'] = 'grey'  
    df.loc[(df['padj'] < significance_threshold), 'color'] = 'grey'  
    df.loc[(df['log2FoldChange'] >= fold_change_threshold) & (df['padj'] > significance_threshold), 'color'] = 'red' # Up-regolati
    df.loc[(df['log2FoldChange'] <= -fold_change_threshold) & (df['padj'] > significance_threshold), 'color'] = 'blue'  # Down-regolati
   
    fig=go.Figure()
    trace1=go.Scatter(
        x=df['log2FoldChange'],
        y=df['padj'],
        mode='markers',
        hovertext=list(df['GeneSymbol']),
        marker=dict(
            color=df['color'],  # Usa la colonna color per colorare i punti
            size=10
        ),
    )
    fig.add_trace(trace1)
    fig.add_hline(y=(significance_threshold),line_dash="dash")# Padj cutoff
    fig.add_vline(x=fold_change_threshold, line_dash="dash")# line_color="black")  # Fold change positivo cutoff
    fig.add_vline(x=-fold_change_threshold, line_dash="dash")#, line_color="black")  # Fold change negativo cutoff

    fig.update_layout(
        title="Volcano Plot",
        xaxis_title="log2 Fold Change",
        yaxis_title="-log10(padj)"
    )
    #fig.write_html(cartella+'/'+tumor+'.html')
    fig.show()

if __name__ == "__main__":
    # tumor= sys.argv[1]
    # file_result= sys.argv[2] #"/mnt/data/notturno/deseq2/tobacco_smoking_history/BLCA/result_BLCA.txt"
    # dir_saveresults=sys.argv[3]
    tumor='CHOL'
    file_result= "res_CHOL_KRAS.txt"
    dir_saveresults="/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova/"
    #generare file html plotly per volcano plot nella cartella risultati
    path_result=os.path.join(dir_saveresults,file_result)
    print(path_result)
    df=pd.read_csv(path_result,sep="\t",header=None,skiprows=1)

    df.columns=["ENSG","baseMean","log2FoldChange","lfcSE","stat","pvalue","padj"]

    file_mapped=mapping_ensg()
    ens2sym=pd.read_csv(file_mapped,sep="\t",index_col=0)
    

    GS=[]
    for i in df.ENSG:
        if i in ens2sym.index:
            GS.append(ens2sym.loc[i,"gene_symbol"])
        else:
            GS.append(i)


    df["GeneSymbol"]=GS
    col=["GeneSymbol","ENSG","log2FoldChange","padj"]

    df=df.loc[:,col].set_index("ENSG")
  
    df.log2FoldChange=[round(x, 3) for x in df.log2FoldChange]

    #df.to_csv(os.path.join(dir_saveresults,'result.txt'), sep="\t")

    display(df.head())
    print('################')
    #df_res=pd.read_csv(os.path.join(dir_saveresults,'result.txt'),sep="\t")
   # display(df_res.head())
    
    df_plot=df.copy()
    df_plot['padj']=np.log10(df_plot['padj'])*(-1)
    #df_plot.to_csv(os.path.join(dir_saveresults,'result_-log.txt'), sep="\t")
    display(df_plot.head())
    plotly_volcano(df_plot,dir_saveresults,tumor)
    

    # df_res.padj=['%.2E' % Decimal(x) for x in df_res.padj]
    # display(df_res)

    #df.padj=['%.2E' % Decimal(x) for x in df.padj]
    display(df)


    display(df.sort_values(by='padj', ascending=True).head(10))
    # #plt
    # df_plot=df
    # #df_plot['padj']=np.log10(df_plot['padj'])*(-1)
    # df['padj']=np.log10(df['padj'])*(-1)
    # plotly_volcano(df,dir_saveresults,tumor)

    # #sovrascrive il file di risultati
    # df.padj=['%.2E' % Decimal(x) for x in df.padj]
     
    
    # #df.to_csv(path_result,sep="\t")
    # #df.to_csv(os.path.join(dir_saveresults,'result.txt'), sep="\t")
